In [48]:
import pickle
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

FILE_DIR = 'sub_model_output'
DATA_PATH = '.'

# Read

In [49]:
df_logit_list = []
for name in os.listdir(FILE_DIR):
    if 'mask' in name or 'ipynb_checkpoints' in name:
        continue
    df_logit = pickle.load(open('%s/%s'%(FILE_DIR,name), 'rb'))
    df_logit_list.append(df_logit)
df_train_origin_weibo = pd.read_csv('%s/train.origin_weibo.csv'%DATA_PATH, sep='\t')
df_train_origin_weibo.index = df_train_origin_weibo.WeiboId

In [50]:
for i in range(len(df_logit_list)):
    print(sum(df_logit_list[0]['WeiboId'] == df_logit_list[i]['WeiboId']))

20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329
20329


In [51]:
def precision_score_cch(predictions, ground_truths):
    
    predictions, ground_truths = np.array(predictions)-1, np.array(ground_truths)-1
    y_pred = predictions
    y_true = ground_truths

    w=[1,10,50,100,300]
    n = len(y_true)
    count_r = [0 for i in range(5)]
    count = [0 for i in range(5)]
    for i in range(n):
        count[y_true[i]] += 1
        if y_pred[i] == y_true[i]:
            count_r[y_pred[i]] += 1
    sum1 = sum(w[i]*count_r[i] for i in range(5))
    sum2 = sum(w[i]*count[i] for i in range(5))
    precision = sum1/sum2
    return precision

def df_logit2score(df_logit, ground_truths):
    logit_name = sorted(list(set(df_logit.columns)-set(['WeiboId'])))
    logit = np.array(df_logit[logit_name])
    if len(logit_name) == 5:
        predictions = np.argmax(np.array(logit), 1)
    elif len(logit_name) == 1:
        predictions = np.array([count2idx(num) for num in logit[:, 0]])
    else:
        return -1
    cnt = 0
    for p, g in zip(predictions, ground_truths):
        if p+1 == g:
            cnt += 1
    return precision_score_cch(predictions+1, ground_truths), cnt/predictions.shape[0]

In [52]:
def count2idx(num):
    if 0 <= num <= 10:
        return 0
    elif 10 <= num <= 50:
        return 1
    elif 50 <= num <= 150:
        return 2
    elif 150 <= num <= 300:
        return 3
    elif num > 300:
        return 4
    return 0
    
df_train_origin_weibo = pd.read_csv('%s/train.origin_weibo.csv'%DATA_PATH, sep='\t')
df_train_origin_weibo.index = df_train_origin_weibo.WeiboId


group_truths = np.array(df_train_origin_weibo['ForwordCount'].apply(count2idx)) + 1
df_logit_list = []
feature_name_list = []
for name in os.listdir(FILE_DIR):
    if 'mask' in name or 'ipynb_checkpoints' in name:
        continue

    df_logit = pickle.load(open('%s/%s'%(FILE_DIR,name), 'rb'))
    logit_name = list(set(df_logit.columns)-set(['WeiboId']))
    df_logit = df_logit[logit_name]
    logit_name = ['%s_%s'%(name.split('.')[0], f) for f in logit_name]
    df_logit.columns = logit_name
    feature_name_list.extend(logit_name)
    df_logit_list.append(df_logit)
    print(name, df_logit2score(df_logit, group_truths))

df_nn_loss_soft_ce.pickle (0.6711899287746997, 0.6136553691770378)
df_nn_loss_jumpce_jump_ce.pickle (0.6306574272071, 0.5210290717693935)
df_nn_loss_mse.pickle (0.6198852034664051, 0.5709085542820601)
df_nn_loss_soft_ce_10_fold.pickle (0.6840147702729499, 0.6208372276058832)
cch_xgb_old.pickle (0.6800944311354782, 0.5394264351419155)
df_lgb_yjq.pickle (0.6257080995332036, 0.6677652614491613)
df_nn_bert_loss_mse.pickle (0.6154101751960169, 0.6040139701903684)
cch_xgb.pickle (0.6861156218681501, 0.5826651581484579)
cch_lgb.pickle (0.6914936947655568, 0.5942249987702297)
df_nn_bert_loss_ce.pickle (0.6610313466351538, 0.5822224408480496)
cch_xgbReg.pickle (0.6439190529018013, 0.6626494170888878)
df_lgb.pickle (0.6859977169316848, 0.5905848787446505)
df_nn_bert_loss_poisson.pickle (0.6323214946058492, 0.36676668798268486)
cch_lgbReg.pickle (0.6053132251823508, 0.6600423040975946)
df_nn_loss_ce.pickle (0.6713078337111651, 0.5902897338777117)
df_nn_loss_poisson.pickle (0.6175431826829805, 0.3

In [53]:
# df_LSTM_yjq = pickle.load(open('%s/df_LSTM_yjq.pickle'%(FILE_DIR), 'rb'))
# df_LSTM_yjq = pd.concat([df_LSTM_yjq[['WeiboId']],pd.get_dummies(df_LSTM_yjq['LSTM_pre'])], 1)
# pickle.dump(df_LSTM_yjq, open('%s/df_LSTM_yjq.pickle'%(FILE_DIR), 'wb'))

In [54]:
# pickle.load(open('%s/cch_lgbReg.pickle'%(FILE_DIR), 'rb'))['lgbReg_cch_fowardcount'].apply(count2idx).value_counts(1)

In [55]:
df_logits = pd.concat(df_logit_list, 1)
df_logits.index = pickle.load(open('sub_model_output/df_lgb.pickle', 'rb')).WeiboId
feature_name_list = list(df_logits.columns)
df_logits['label'] = df_train_origin_weibo['ForwordCount'].apply(count2idx)
df_logits['type'] = ['train'] * 18000 + ['test'] * 2329

/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


In [56]:
def get_train_test_from_df(df, feature_name_list):
    df_train = df.iloc[:18000]
    df_test = df.iloc[18000:]
    train_x_fix, train_y, test_x_fix = \
        np.array(df_train[feature_name_list]), np.array(df_train['label']), np.array(df_test[feature_name_list])
    
        
    return train_x_fix, train_y, test_x_fix

train_x, train_y, test_x = \
get_train_test_from_df(df_logits, feature_name_list)

train_x.shape, train_y.shape, test_x.shape

((18000, 71), (18000,), (2329, 71))

# 权重赋予

In [57]:
from sklearn.metrics import roc_auc_score

def fit_binary_model(train_x, train_y):
    trn_data = lgb.Dataset(train_x, train_y)
    clf = lgb.train({  
                'boosting_type': 'gbdt',  
                'objective': 'binary',  
                'metric': 'auc',  
                'num_leaves': 8,  
                'max_depth': 3,
                'min_data_in_leaf': 100,  
                'learning_rate': 0.06,  
                'feature_fraction': 0.8,  
                'bagging_fraction': 0.8,  
                'bagging_freq': 5,  
                'lambda_l1': 0.5,  
                'lambda_l2': 0.5,  
                'min_gain_to_split': 0.2,  
                'verbose': -1, 
                }, 
                trn_data, 
                num_boost_round = 100,
                valid_sets = [trn_data], 
                verbose_eval = 100, 
                early_stopping_rounds = 100)
    return clf

def eval_logit(logit, label):
    return roc_auc_score(label, logit)
    
def cross_validation_fit_weight(train_x, train_y):
    n_flod = 5
    folds = KFold(n_splits=n_flod, shuffle=True)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    score_train = np.zeros((len(train_x), ))
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
        
        trn_x, trn_labels = train_x[trn_idx], train_y[trn_idx]
        val_x, val_labels = train_x[val_idx], train_y[val_idx]
        model = fit_binary_model(trn_x, trn_labels)
        score_train[val_idx] = model.predict(val_x)
        
        print(eval_logit(model.predict(trn_x), trn_labels))
        print(eval_logit(score_train[val_idx], val_labels))
    
    print(eval_logit(score_train, train_y))
    return score_train

def fit_train_test_weight(train_x, test_x):
    train_y = np.array([0]*train_x.shape[0] + [1]*test_x.shape[0])
    return cross_validation_fit_weight(np.concatenate([train_x, test_x]), train_y)


dataset_weights = fit_train_test_weight(train_x, test_x)

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.893306
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.893306
0.8933063058784899
0.8554931282905416


/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.898343
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.898343
0.8983427276727406
0.8843503561153035


/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.891327
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.891327
0.8913266382203039
0.8772108695885034


/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.901005
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.901005
0.9010049141200437
0.881289544227859


/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.895779
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.895779
0.8957786911373702
0.8819658798365717
0.8760438791088212


# 训练

In [58]:
SEED = 666 
# import random
# SEED = random.randint(0, 1314)
# print(SEED)

params = {  
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 5,  
    'metric': 'multi_error',  
    'num_leaves': 16,  
    'max_depth': 4,
    'min_data_in_leaf': 100,  
    'learning_rate': 0.005,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.5,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1, 
    'feature_fraction_seed':SEED,
    'bagging_seed':SEED,

} 

def precision_score_cch(predictions, ground_truths):
    
    predictions, ground_truths = np.array(predictions)-1, np.array(ground_truths)-1
    y_pred = predictions
    y_true = ground_truths

    w=[1,10,50,100,300]
    n = len(y_true)
    count_r = [0 for i in range(5)]
    count = [0 for i in range(5)]
    for i in range(n):
        count[y_true[i]] += 1
        if y_pred[i] == y_true[i]:
            count_r[y_pred[i]] += 1
    sum1 = sum(w[i]*count_r[i] for i in range(5))
    sum2 = sum(w[i]*count[i] for i in range(5))
    precision = sum1/sum2
    return precision

def fit_lgb(train_x, train_y, sub_dataset_weights):
    class_weights = [1,10,50,100,300]
#     sub_dataset_weights = [1] * len(sub_dataset_weights)
    trn_data = lgb.Dataset(train_x, train_y,
                           weight=[class_weights[int(y)]*sub_dataset_weights[i] for i, y in enumerate(train_y)])
    clf = lgb.train(params, 
                    trn_data, 
                    num_boost_round = 400,
                    valid_sets = [trn_data], 
                    verbose_eval = 100, 
                    early_stopping_rounds = 100
                   )
    return clf

def eval_logit(logit, label):
    prediction = np.argmax(logit, 1)
    return precision_score_cch(prediction+1, np.array(label).astype('int')+1)
    
def cross_validation_lgb(train_x, train_y, test_x, dataset_weights):
    n_flod = 25
    folds = KFold(n_splits=n_flod, shuffle=True, random_state=SEED)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    score_train = np.zeros((len(train_x), 5))
    score_test = np.zeros((len(test_x), 5))
    train_score_list = []
    for flod, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
        
        trn_x, trn_labels = train_x[trn_idx], train_y[trn_idx]
        val_x, val_labels = train_x[val_idx], train_y[val_idx]
        model = fit_lgb(trn_x, trn_labels, dataset_weights[trn_idx])
        score_train[val_idx] = model.predict(val_x)
        score_test += model.predict(test_x)/n_flod
        
        train_score = eval_logit(model.predict(trn_x), trn_labels)
        print(flod)
        print(train_score)
        train_score_list.append(train_score)
        print(eval_logit(score_train[val_idx], val_labels))
    
    prediction = np.argmax(score_train, 1)
    val_score = precision_score_cch(prediction+1, np.array(train_y).astype('int')+1)
    
    print('train score %f, val score %f'%(np.mean(train_score_list), val_score))

    return score_train, score_test

score_train, score_test = cross_validation_lgb(train_x, train_y, test_x, dataset_weights)

/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/huangweilin/anaconda3/envs/hyr2/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	training's multi_error: 0.275751
[200]	training's multi_error: 0.258379
[300]	training's multi_error: 0.250814
[400]	training's multi_error: 0.2428
Did not meet early stopping. Best iteration is:
[380]	training's multi_error: 0.241968
0
0.7568334632284953
0.6616012104466812
Training until validation scores don't improve for 100 rounds
[100]	training's multi_error: 0.282398
[200]	training's multi_error: 0.266302
[300]	training's multi_error: 0.253433
[400]	training's multi_error: 0.247801
Did not meet early stopping. Best iteration is:
[400]	training's multi_error: 0.247801
1
0.7510474577313421
0.7940432158847577
Training until validation scores don't improve for 100 rounds
[100]	training's multi_error: 0.281138
[200]	training's multi_error: 0.262634
[300]	training's multi_error: 0.252826
[400]	training's multi_error: 0.247253
Did not meet early stopping. Best iteration is:
[395]	training's multi_error: 0.246623
2
0.752

23
0.7588181087387016
0.6473609306354805
Training until validation scores don't improve for 100 rounds
[100]	training's multi_error: 0.279861
[200]	training's multi_error: 0.260179
[300]	training's multi_error: 0.251431
[400]	training's multi_error: 0.240715
Did not meet early stopping. Best iteration is:
[390]	training's multi_error: 0.240426
24
0.7583236909211487
0.6665103556076593
train score 0.755969, val score 0.694380


In [44]:
# sub_feature_name = ['lgb_hyr_0', 'lgb_hyr_1', 'lgb_hyr_2', 'lgb_hyr_3','lgb_hyr_4']
# sub_feature_name =  ['nn_hyr_loss_ce_0', 'nn_hyr_loss_ce_1', 'nn_hyr_loss_ce_2', 'nn_hyr_loss_ce_3', 'nn_hyr_loss_ce_4']
    
# prediction = np.argmax(np.array(df_logits[sub_feature_name]), 1)
prediction = np.argmax(score_train, 1)
print(pd.Series(prediction).value_counts(1))
print('baseline', precision_score_cch(prediction+1, np.array(train_y).astype('int')+1))

1    0.456611
0    0.319444
2    0.163167
4    0.033222
3    0.027556
dtype: float64
baseline 0.699173593581684


In [45]:
prediciton_test = np.argmax(score_test, 1)
df_submit = pd.DataFrame({
    'WeiboId':list(df_logits.query('type=="test"').index),
    'ForwardScale':prediciton_test+1
})
SUBMIT_PATH = '.'
df_submit.to_csv('%s/submission.csv'%SUBMIT_PATH, sep='\t', index=False)

In [68]:
df_submit = pd.read_csv('%s/submission.csv'%SUBMIT_PATH,sep='\t')

In [69]:
df_submit_old = pd.read_csv('submit/submission_12_12.csv', sep='\t')

In [70]:
cnt = 0
for a, b in zip(df_submit['ForwardScale'], df_submit_old['ForwardScale']):
    if a==b:
        cnt += 1
        
print(cnt/2329)

0.9540575354229283
